In [ ]:
# Import de bibliothèques

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras

In [ ]:
# Import des données (sur le site du gouvernement)
data = pd.read_csv('/content/WHO-COVID-19-global-data.csv')

print(f'Le dataset contient {data.shape[0]} lignes et {data.shape[1]} colonnes.\n')

nb_pays = len(data[' Country'].unique())
print(f'Pour un total de {nb_pays} pays.\n')

col = ', '.join(list(data.columns))
print(f'Noms de colonnes : {col}\n')

data

Le dataset contient 32524 lignes et 8 colonnes.

Pour un total de 216 pays.

Noms de colonnes : Date_reported,  Country_code,  Country,  WHO_region,  New_cases,  Cumulative_cases,  New_deaths,  Cumulative_deaths



,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-02-24,AF,Afghanistan,EMRO,5,5,0,0
1,2020-02-25,AF,Afghanistan,EMRO,0,5,0,0
2,2020-02-26,AF,Afghanistan,EMRO,0,5,0,0
3,2020-02-27,AF,Afghanistan,EMRO,0,5,0,0
4,2020-02-28,AF,Afghanistan,EMRO,0,5,0,0
...,...,...,...,...,...,...,...,...
32519,2020-07-30,ZW,Zimbabwe,AFRO,62,2879,1,41
32520,2020-07-31,ZW,Zimbabwe,AFRO,213,3092,12,53
32521,2020-08-01,ZW,Zimbabwe,AFRO,77,3169,14,67
32522,2020-08-02,ZW,Zimbabwe,AFRO,490,3659,2,69


In [ ]:
# On garde seulement les variables nécessaires à l'analyse

data.columns = [col.strip() for col in data.columns]
data = data.drop(['Country_code','WHO_region'], axis=1)
data.groupby(['Country']).sum().sort_values(by=['New_deaths'], ascending=False).head(10)

,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
Country,,,,
United States of America,4582276,245003407,153757,11609377
Brazil,2707877,102519720,93563,4392551
Mexico,434193,17032703,47472,1959403
The United Kingdom,304699,28186842,46201,4265869
India,1803695,50897974,38135,1328256
Italy,248070,28282599,35154,3923364
France,175920,17762116,30147,3221562
Spain,288522,29216360,28445,3310314
Peru,422183,21316636,19408,751669


In [ ]:
# Vérification des données du dataset avec une recherche Google pour le 4 avril 2020
data[(data['Country'] == 'France') & (data['Date_reported'] == '2020-04-04')]

,Date_reported,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
10377,2020-04-04,France,5209,63536,2003,6493


In [ ]:
# Visualisation des décès quotidients pour chaque pays

fig = go.Figure()

fig.add_trace(go.Scatter(x=data[data['Country'] == 'France']['Date_reported'], y=data[data['Country'] == 'France']['New_deaths'], mode='lines', name="France"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'United States of America']['Date_reported'], y=data[data['Country'] == 'United States of America']['New_deaths'], mode='lines', name="USA"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'The United Kingdom']['Date_reported'], y=data[data['Country'] == 'The United Kingdom']['New_deaths'], mode='lines', name="UK"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'Italy']['Date_reported'], y=data[data['Country'] == 'Italy']['New_deaths'], mode='lines', name="Italie"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'Mexico']['Date_reported'], y=data[data['Country'] == 'Mexico']['New_deaths'], mode='lines', name="Mexique"))

fig.add_trace(go.Scatter(
    x=['2020-03-17'],
    y=[-500],
    text="Début confinement France : 17 mars 2020",
    mode="text",
    showlegend=False))

fig.add_trace(go.Scatter(
    x=['2020-05-11'],
    y=[-500],
    text="Fin confinement France : 11 mai 2020",
    mode="text",
    showlegend=False))

fig.update_layout(shapes=[dict(type= 'line',
                               line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                               yref= 'paper', y0=0, y1=1,
                               # yref= 'y', y0=-100, y1=1000, 
                               xref= 'x', x0='2020-03-17', x1='2020-03-17'),
                          
                          dict(type= 'line',
                               line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                               yref= 'paper', y0=0, y1=1,
                               # yref= 'y', y0=-100, y1=1000, 
                               xref= 'x', x0='2020-05-11', x1='2020-05-11')              
                          ])

fig.update_layout(title='Évolution décès dû au Covid-19',
                   xaxis_title='Temps',
                   yaxis_title='Nombre décès quotidien')

In [ ]:
# Même visualisation mais pour le nombre de décès cumulés

fig = go.Figure()

fig.add_trace(go.Scatter(x=data[data['Country'] == 'France']['Date_reported'], y=data[data['Country'] == 'France']['Cumulative_deaths'], mode='lines', name="France"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'United States of America']['Date_reported'], y=data[data['Country'] == 'United States of America']['Cumulative_deaths'], mode='lines', name="USA"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'The United Kingdom']['Date_reported'], y=data[data['Country'] == 'The United Kingdom']['Cumulative_deaths'], mode='lines', name="UK"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'Italy']['Date_reported'], y=data[data['Country'] == 'Italy']['Cumulative_deaths'], mode='lines', name="Italie"))
fig.add_trace(go.Scatter(x=data[data['Country'] == 'Mexico']['Date_reported'], y=data[data['Country'] == 'Mexico']['Cumulative_deaths'], mode='lines', name="Mexique"))

fig.add_trace(go.Scatter(
    x=['2020-03-17'],
    y=[-500],
    text="Début confinement France : 17 mars 2020",
    mode="text",
    showlegend=False))

fig.add_trace(go.Scatter(
    x=['2020-05-11'],
    y=[-500],
    text="Fin confinement France : 11 mai 2020",
    mode="text",
    showlegend=False))

fig.update_layout(shapes=[dict(type= 'line',
                               line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                               yref= 'paper', y0=0, y1=1,
                               # yref= 'y', y0=-100, y1=1000, 
                               xref= 'x', x0='2020-03-17', x1='2020-03-17'),
                          
                          dict(type= 'line',
                               line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                               yref= 'paper', y0=0, y1=1,
                               # yref= 'y', y0=-100, y1=1000, 
                               xref= 'x', x0='2020-05-11', x1='2020-05-11')              
                          ])

fig.update_layout(title='Évolution des décès cumulés dû au Covid-19',
                   xaxis_title='Temps',
                   yaxis_title='Nombre décès cumulés')

In [ ]:
# Obtention de la population de chaque pays pour pouvoir faire une analyse plus fine

pop = pd.read_csv('/content/API_SP.POP.TOTL_DS2_en_csv_v2_1217749.csv', skiprows=3)

print(f'Le dataset contient {pop.shape[0]} lignes et {pop.shape[1]} colonnes.\n')

nb_pays = len(pop['Country Code'].unique())
print(f'Pour un total de {nb_pays} pays.\n')

col = ', '.join(list(pop.columns))
print(f'Noms de colonnes : {col}\n')

pop

Le dataset contient 264 lignes et 65 colonnes.

Pour un total de 264 pays.

Noms de colonnes : Country Name, Country Code, Indicator Name, Indicator Code, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, Unnamed: 64



,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,76700.0,80324.0,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,10174836.0,10399926.0,10637063.0,10893776.0,11173642.0,11475445.0,11791215.0,12108963.0,12412950.0,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,13171673.0,12882528.0,12537730.0,12204292.0,11938208.0,11736179.0,11604534.0,11618005.0,11868877.0,12412308.0,13299017.0,14485546.0,15816603.0,17075727.0,18110657.0,18853437.0,19357126.0,19737765.0,20170844.0,20779953.0,21606988.0,22600770.0,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,5781214.0,5774243.0,5771652.0,5803254.0,5890365.0,6040777.0,6248552.0,6496962.0,6761380.0,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,8640446.0,8952950.0,9278096.0,9614754.0,9961997.0,10320111.0,10689250.0,11068050.0,11454777.0,11848386.0,12248901.0,12657366.0,13075049.0,13503747.0,13945206.0,14400719.0,14871570.0,15359601.0,15866869.0,16395473.0,16945753.0,17519417.0,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,52448.0,54509.0,56671.0,58888.0,60971.0,62677.0,63850.0,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,1106000.0,1135000.0,1163000.0,1191000.0,1219000.0,1247000.0,1278000.0,1308000.0,1339000.0,1369000.0,1400000.0,1430000.0,1460000.0,1491000.0,1521000.0

In [ ]:
# Ajout de la variable population

pop['Country'] = pop['Country Name']
pop = pop[['Country','2019']]

data.replace('United States of America','United States', inplace=True)
data.replace('The United Kingdom','United Kingdom', inplace=True)

jointure = pd.merge(data, pop, on='Country')
jointure[jointure['Country'] == 'France']

,Date_reported,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,2019
8741,2020-01-24,France,7,7,0,0,67059887.0
8742,2020-01-25,France,0,7,0,0,67059887.0
8743,2020-01-26,France,0,7,0,0,67059887.0
8744,2020-01-27,France,0,7,0,0,67059887.0
8745,2020-01-28,France,1,8,0,0,67059887.0
...,...,...,...,...,...,...,...
8929,2020-07-30,France,1307,173455,15,30120,67059887.0
8930,2020-07-31,France,1193,174648,16,30136,67059887.0
8931,2020-08-01,France,1272,175920,11,30147,67059887.0
8932,2020-08-02,France,0,175920,0,30147,67059887.0


In [ ]:
# Liste par pays du nombre de morts pour 1 000 000 habitants (ordre décroissant)

jointure['Cumulative_deathsPer1M'] = (jointure['Cumulative_deaths'] / jointure['2019'])*1000000
liste_pays = jointure[jointure['Date_reported'] == '2020-07-23'].sort_values(by='Cumulative_deathsPer1M', ascending=False).head(10)['Country']
jointure[jointure['Date_reported'] == '2020-07-23'].sort_values(by='Cumulative_deathsPer1M', ascending=False).head(10)['Country']

20900        San Marino
2641            Belgium
24133    United Kingdom
613             Andorra
22692             Spain
12728             Italy
23353            Sweden
5587              Chile
8922             France
25673     United States
Name: Country, dtype: object

In [ ]:
# Autres visualisation : nombre de morts cumulés pour 1 000 000 d'habitants 

fig = go.Figure()

for pays in liste_pays:
  print()
  fig.add_trace(go.Scatter(x=jointure[jointure['Country'] == pays]['Date_reported'], y=jointure[jointure['Country'] == pays]['Cumulative_deathsPer1M'], mode='lines', name=pays))

  fig.add_trace(go.Scatter(
      x=['2020-03-17'],
      y=[-80],
      text="Début confinement France : 17 mars 2020",
      mode="text",
      showlegend=False))

  fig.add_trace(go.Scatter(
      x=['2020-05-11'],
      y=[-80],
      text="Fin confinement France : 11 mai 2020",
      mode="text",
      showlegend=False))

  fig.update_layout(shapes=[dict(type= 'line',
                                line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                                yref= 'paper', y0=0, y1=1,
                                # yref= 'y', y0=-100, y1=1000, 
                                xref= 'x', x0='2020-03-17', x1='2020-03-17'),
                            
                            dict(type= 'line',
                                line=dict(width=0.5), # line=dict(color='firebrick', width=0.5)
                                yref= 'paper', y0=0, y1=1,
                                # yref= 'y', y0=-100, y1=1000, 
                                xref= 'x', x0='2020-05-11', x1='2020-05-11')              
                            ])

  fig.update_layout(title='Évolution décès cumulés dus au Covid-19 pour un million d\'habitants',
                    xaxis_title='Temps',
                    yaxis_title='Nombre décès quotidien pour un million d\'habitants')
  
fig.show()

Prédiction chute

In [ ]:
# Création des fenêtre de 60 jours (30 jours d'historique et 30 jours de prédiction)

from sklearn.preprocessing import StandardScaler

X_train_all, y_train_all, X_validation_all, y_validation_all, X_test_all, y_test_all = [], [], [], [], [], []
liste_encode, liste_pays = [], []

countries = data.groupby(['Country']).sum().sort_values(by=['New_deaths'], ascending=False)

for indice, pays in enumerate(countries[countries['New_deaths'] != 0].index):
  try:
    print(pays)
    data_france = data[data['Country'] == pays]

    data_france.index = data_france['Date_reported']
    data_france.drop(['Date_reported','Country','Cumulative_cases'], axis=1, inplace=True)

    split = 20 # 20 : nombre de j pour le jeu d'entrainement
    nb_test = 106 # 1 # 106 : nb de jours avant le dernier jours du dataset (c'est-à-dire à partir du 20 avril 2020)
                 # le jeu de validation est de taille : nb jours dataset - split - nb_test

    nb_historique = 30 # 30 : nb de jours d'historique qu'on donne pour prédire
    nb_prediction = 30 # 30 : nb de jours que l'on souhaite prédire

    # Variable à prédire -> New_deaths
    encode_autre = StandardScaler()
    data_france[['New_cases','Cumulative_deaths']] = pd.DataFrame(encode_autre.fit_transform(data_france[['New_cases','Cumulative_deaths']]),
                                                                    index=data_france[['New_cases','Cumulative_deaths']].index,
                                                                    columns=data_france[['New_cases','Cumulative_deaths']].columns)

    encode_pred = StandardScaler()
    data_france['New_deaths'] = pd.DataFrame(encode_pred.fit_transform(np.array(data_france['New_deaths']).reshape(-1, 1)),
                                                                    index=data_france['New_deaths'].index,
                                                                    columns=pd.DataFrame(data_france['New_deaths']).columns)
    liste_encode.append(encode_pred)
 
    # Création des fenêtres
    liste_data_fenetre = [np.array(data_france.iloc[i-nb_historique-nb_prediction:i,:][['New_deaths','New_cases','Cumulative_deaths']]) for i in range(nb_historique+nb_prediction,data_france.shape[0])]

    print(data_france.shape[0])
    print(len(liste_data_fenetre))
    print(liste_data_fenetre[0].shape)

    X_train = np.array([liste_data_fenetre[:split][i][:nb_historique] for i in range(split)]).reshape(-1,nb_historique,3)
    y_train =  np.array([liste_data_fenetre[:split][i][nb_historique:,0] for i in range(split)]).reshape(-1,nb_prediction)

    X_validation = np.array([liste_data_fenetre[split:len(liste_data_fenetre)-nb_test][i][:nb_historique] for i in range(len(liste_data_fenetre)-split-nb_test)]).reshape(-1,nb_historique,3)
    y_validation = np.array([liste_data_fenetre[split:len(liste_data_fenetre)-nb_test][i][nb_historique:,0] for i in range(len(liste_data_fenetre)-split-nb_test)]).reshape(-1,nb_prediction)

    X_test = np.array([liste_data_fenetre[len(liste_data_fenetre)-nb_test:][i][:nb_historique] for i in range(nb_test)]).reshape(-1,nb_historique,3)
    y_test = np.array([liste_data_fenetre[len(liste_data_fenetre)-nb_test:][i][nb_historique:,0] for i in range(nb_test)]).reshape(-1,nb_prediction)

    X_train_all.append(X_train)
    y_train_all.append(y_train)
    X_validation_all.append(X_validation)
    y_validation_all.append(y_validation)
    X_test_all.append(X_test)
    y_test_all.append(y_test)

    liste_pays.append(pays)
  except:
    continue

United States
197
137
(60, 3)
Brazil


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

160
100
(60, 3)
Mexico
158
98
(60, 3)
United Kingdom
185
125
(60, 3)
India


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

187
127
(60, 3)
Italy
188
128
(60, 3)
France


...

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

125
65
(60, 3)
Curaçao
144
84
(60, 3)
Montserrat
139
79
(60, 3)
Monaco


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

157
97
(60, 3)
Cayman Islands
144
84
(60, 3)
British Virgin Islands
131
71
(60, 3)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [ ]:
for i in range(1, len(X_train_all)):
  X_train_all[0] = np.append(X_train_all[0], X_train_all[i], axis=0)

for i in range(1, len(y_train_all)):
  y_train_all[0] = np.append(y_train_all[0], y_train_all[i], axis=0)

for i in range(1, len(X_validation_all)):
  X_validation_all[0] = np.append(X_validation_all[0], X_validation_all[i], axis=0)

for i in range(1, len(y_validation_all)):
  y_validation_all[0] = np.append(y_validation_all[0], y_validation_all[i], axis=0)

print(f'Train : {X_train_all[0].shape}, {y_train_all[0].shape}, Validation : {X_validation_all[0].shape}, {y_validation_all[0].shape}')
print('Exemples pour certains pays :')
print(X_train_all[1].shape, y_train_all[1].shape, X_validation_all[1].shape, y_validation_all[1].shape)
print(X_train_all[2].shape, y_train_all[2].shape, X_validation_all[2].shape, y_validation_all[2].shape)
print(X_train_all[3].shape, y_train_all[3].shape, X_validation_all[3].shape, y_validation_all[3].shape)
print(X_train_all[4].shape, y_train_all[4].shape, X_validation_all[4].shape, y_validation_all[4].shape)

Train : (3000, 30, 3), (3000, 30), Validation : (260, 30, 3), (260, 30)
Exemples pour certains pays :
(120, 30, 3) (120, 30) (4, 30, 3) (4, 30)
(120, 30, 3) (120, 30) (6, 30, 3) (6, 30)
(120, 30, 3) (120, 30) (7, 30, 3) (7, 30)
(120, 30, 3) (120, 30) (12, 30, 3) (12, 30)


In [ ]:
# Création et entraînement du modèle RNN

model = keras.models.Sequential([
    keras.layers.LSTM(60, return_sequences=True, input_shape=[nb_historique, 3]),
    keras.layers.LSTM(30, activation='relu'),
    keras.layers.Dense(nb_prediction)
])

model.compile(loss="mse", 
              optimizer="adam",
              metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_mae',
                                              patience=10, 
                                              restore_best_weights=True)

history = model.fit(X_train_all[0], y_train_all[0],
                    epochs=200,
                    validation_data=(X_validation_all[0], y_validation_all[0]),
                    callbacks=[early_stopping]) 

Epoch 1/200
94/94 [==============================] - 3s 31ms/step - loss: 0.9043 - mae: 0.5990 - val_loss: 0.4459 - val_mae: 0.3412
Epoch 2/200
94/94 [==============================] - 2s 25ms/step - loss: 0.7059 - mae: 0.4930 - val_loss: 0.4288 - val_mae: 0.2952
Epoch 3/200
94/94 [==============================] - 2s 26ms/step - loss: 0.6742 - mae: 0.4655 - val_loss: 0.4119 - val_mae: 0.2791
Epoch 4/200
94/94 [==============================] - 2s 26ms/step - loss: 0.6553 - mae: 0.4521 - val_loss: 0.4080 - val_mae: 0.2759
Epoch 5/200
94/94 [==============================] - 2s 26ms/step - loss: 0.6529 - mae: 0.4497 - val_loss: 0.4094 - val_mae: 0.2813
Epoch 6/200
94/94 [==============================] - 2s 26ms/step - loss: 0.6293 - mae: 0.4327 - val_loss: 0.4125 - val_mae: 0.2784
Epoch 7/200
94/94 [==============================] - 2s 26ms/step - loss: 0.6224 - mae: 0.4295 - val_loss: 0.4088 - val_mae: 0.2870
Epoch 8/200
94/94 [==============================] - 2s 25ms/step - loss: 0.

In [ ]:
# Performance du modèle par pays

for i,pays in enumerate(liste_pays):
  print(i, pays)
  model.evaluate(X_test_all[i], y_test_all[i])

0 United States
1/1 [==============================] - 0s 1ms/step - loss: 0.2227 - mae: 0.4069
1 United Kingdom
1/1 [==============================] - 0s 3ms/step - loss: 0.0261 - mae: 0.1411
2 India
1/1 [==============================] - 0s 1ms/step - loss: 0.6223 - mae: 0.6327
3 Italy
1/1 [==============================] - 0s 2ms/step - loss: 0.0089 - mae: 0.0856
4 France
1/1 [==============================] - 0s 1ms/step - loss: 0.0057 - mae: 0.0567
5 Spain
1/1 [==============================] - 0s 2ms/step - loss: 0.0034 - mae: 0.0518
6 Russian Federation
1/1 [==============================] - 0s 1ms/step - loss: 0.2929 - mae: 0.4615
7 Belgium
1/1 [==============================] - 0s 1ms/step - loss: 0.0030 - mae: 0.0461
8 Germany
1/1 [==============================] - 0s 1ms/step - loss: 0.0139 - mae: 0.1035
9 Canada
1/1 [==============================] - 0s 1ms/step - loss: 0.0209 - mae: 0.1108
10 Sweden
1/1 [==============================] - 0s 1ms/step - loss: 0.1225 - mae: 0

In [ ]:
# Visualisation des prédictions pour les derniers mois

for indice,pays in enumerate(liste_pays):
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=np.array(data[data['Country'] == pays]['Date_reported'])[:-nb_test], y=np.array(data[data['Country'] == pays]['New_deaths'])[:-nb_test], mode='lines', name='Données d\'entraînement et de validation du modèle'))
  fig.add_trace(go.Scatter(x=np.array(data[data['Country'] == pays]['Date_reported'])[-nb_test:], y=np.array(data[data['Country'] == pays]['New_deaths'])[-nb_test:], mode='lines', name='Réalité'))
  fig.add_trace(go.Scatter(x=np.array(data[data['Country'] == pays]['Date_reported'])[-nb_test:], y=liste_encode[indice].inverse_transform(model.predict(X_test_all[indice][-nb_test+nb_historique].reshape(1,nb_historique,3))).reshape(nb_prediction), mode='markers+lines', name=f'{nb_prediction} prédictions d\'après les {nb_historique} jours d\'historique'))

  fig.update_layout(title=f'Évolution des décès dus au Covid-19 | Pays : {pays}',
                      xaxis_title='Temps',
                      yaxis_title='Nombre de décès quotidiens')

  fig.show()